In [1]:
from __future__ import print_function
import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras import utils
import os



Using TensorFlow backend.


In [2]:
workdir = "data/datasets/Project"     
os.chdir(workdir)

In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
np.set_printoptions(edgeitems=10)

In [4]:
def loaddata(path, seed=9527):
    """
    Usage: there should be ./breast/ and ./prostate/ under the path
    """
    
    np.random.seed(seed=seed)
    
    def image2tensor(tumor, cnv):
        train1, train2, train3, testset1, testset2, testset3 = [], [], [], [], [], []
        train = [train1, train2, train3] # 3 channels
        test = [testset1, testset2, testset3]

        rawfnames = !ls ./$tumor/$cnv/*jpg
        rawfnames = np.array(np.sort(rawfnames), dtype=object)
        
        # randomly distribute data to train set and test set in 5:1 ratio
        randomIndices = np.random.permutation(rawfnames.shape[0]//3)
                
        for i, rawf in enumerate(rawfnames):
            im = cv2.imread(rawf, 0)
            channel = i % 3            
            if (randomIndices[i//3]) % 6 == 0:
                test[channel].append(im)
            else:
                train[channel].append(im)
                
        return np.array(train).transpose([1,2,3,0]), np.array(test).transpose([1,2,3,0])
    
    tumors = ["breast", "prostate"]
    cnvs = ["altered", "flat"]
    
    trainset,  testset = {"breast" : {}, "prostate" : {}}, {"breast" : {}, "prostate" : {}}
    data = {"trainset" : trainset , "testset" : testset}
    
    for tumor in tumors:
        for i, cnv in enumerate(cnvs):
            if i == 0:
                trainset[tumor]["pos"], testset[tumor]["pos"] = image2tensor(tumor, cnv)
            elif i == 1:
                trainset[tumor]["neg"], testset[tumor]["neg"] = image2tensor(tumor, cnv)
                
    return data

In [5]:
data = loaddata(workdir)

### Prepare datasets of all cancer types (cropping the cental part)
- label cnv+ breast as 2, cnv+ prostate as 1, cnv- as 0

In [6]:
np.random.seed(seed=5408)

crop = 90 # number of pixels to be cropped from the original image boundary


# training set
x_breast_pos = data["trainset"]["breast"]["pos"][:,crop:-crop,crop:-crop,:] # training set and validation set for cnv+ breast CTC
x_breast_neg = data["trainset"]["breast"]["neg"][:,crop:-crop,crop:-crop,:] # training set and validation set for cnv- breast CTC
x_prostate_pos = data["trainset"]["prostate"]["pos"][:,crop:-crop,crop:-crop,:] # training set and validation set for cnv+ prostate CTC
x_prostate_neg = data["trainset"]["prostate"]["neg"][:,crop:-crop,crop:-crop,:] # training set and validation set for cnv- prostate CTC

x_train = np.concatenate((x_breast_pos, x_prostate_pos, x_breast_neg, x_prostate_neg))
y_train = utils.to_categorical(np.concatenate((2*np.ones(x_breast_pos.shape[0]),
                                               np.ones(x_prostate_pos.shape[0]), 
                                               np.zeros(x_breast_neg.shape[0] + x_prostate_neg.shape[0]))))
# random shuffling
randind = np.random.permutation(y_train.shape[0])
x_train = x_train[randind].astype('float32')
x_train /= 255
y_train = y_train[randind]


# testing set
x_breast_pos = data["testset"]["breast"]["pos"][:,crop:-crop,crop:-crop,:]
x_breast_neg = data["testset"]["breast"]["neg"][:,crop:-crop,crop:-crop,:]
x_prostate_pos = data["testset"]["prostate"]["pos"][:,crop:-crop,crop:-crop,:]
x_prostate_neg = data["testset"]["prostate"]["neg"][:,crop:-crop,crop:-crop,:]

x_test = np.concatenate((x_breast_pos, x_prostate_pos, x_breast_neg, x_prostate_neg)).astype('float32')
x_test /= 255
y_test = utils.to_categorical(np.concatenate((2*np.ones(x_breast_pos.shape[0]), 
                                              np.ones(x_prostate_pos.shape[0]), 
                                              np.zeros(x_breast_neg.shape[0] + x_prostate_neg.shape[0]))))


print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

# Input image dimensions.
input_shape = x_train.shape[1:]

(1547, 45, 45, 3) (1547, 3) (312, 45, 45, 3) (312, 3)


# Trains a ResNet.
ResNet v1:
[Deep Residual Learning for Image Recognition](https://arxiv.org/pdf/1512.03385.pdf)
ResNet v2:
[Identity Mappings in Deep Residual Networks](https://arxiv.org/pdf/1603.05027.pdf)

In [9]:
# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------
n = 3

# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 1

# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

In [10]:
def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder
    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)
    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_v1(input_shape, depth, num_classes=3):
    """ResNet Version 1 Model builder [a]
    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M
    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)
    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


def resnet_v2(input_shape, depth, num_classes=3):
    """ResNet Version 2 Model builder [b]
    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256
    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)
    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


In [11]:
if version == 2:
    model = resnet_v2(input_shape=input_shape, depth=depth)
else:
    model = resnet_v1(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
print(model_type)

Instructions for updating:
Colocations handled automatically by placer.
Learning rate:  0.001
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 45, 45, 3)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 45, 45, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1 (BatchNo (None, 45, 45, 16)   64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 45, 45, 16)   0           batch_normalization_v1[0][0]     
_______________

In [35]:
# Training parameters
batch_size = 32  # orig paper trained all networks with batch_size=128
epochs = 10
data_augmentation = True
num_classes = 3

In [13]:
# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

In [36]:
# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              #validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.1)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    hist = model.fit_generator(
        datagen.flow(x_train, y_train, batch_size=batch_size, subset="training"),
        validation_data=datagen.flow(x_train, y_train, batch_size=batch_size, subset="validation"),
        epochs=epochs,
        use_multiprocessing=True,
        workers=3)
    #model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
     #                   validation_data=(x_test, y_test),
      #                  epochs=epochs, verbose=1, workers=4)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Using real-time data augmentation.
Epoch 1/10
44/44 [==============================] - 64s 1s/step - loss: 0.5834 - acc: 0.8105 - val_loss: 1.0342 - val_acc: 0.7078
Epoch 2/10
44/44 [==============================] - 61s 1s/step - loss: 0.5761 - acc: 0.8349 - val_loss: 0.8930 - val_acc: 0.7143
Epoch 3/10
44/44 [==============================] - 61s 1s/step - loss: 0.5683 - acc: 0.8299 - val_loss: 0.9336 - val_acc: 0.7792
Epoch 4/10
44/44 [==============================] - 61s 1s/step - loss: 0.5149 - acc: 0.8744 - val_loss: 1.0851 - val_acc: 0.7208
Epoch 5/10
44/44 [==============================] - 69s 2s/step - loss: 0.5610 - acc: 0.8385 - val_loss: 0.9354 - val_acc: 0.7662
Epoch 6/10
44/44 [==============================] - 62s 1s/step - loss: 0.5351 - acc: 0.8370 - val_loss: 1.2041 - val_acc: 0.6364
Epoch 7/10
44/44 [==============================] - 61s 1s/step - loss: 0.5400 - acc: 0.8528 - val_loss: 1.1046 - val_acc: 0.7273
Epoch 8/10
44/44 [==============================] - 63s

In [37]:
print(hist.history)

{'loss': [0.584544076392399, 0.5784630656499102, 0.5657696936169889, 0.5173452745772743, 0.5618772627591916, 0.5360480720673374, 0.5389788549150055, 0.5337834855419906, 0.5139873314485734, 0.5128846690326138], 'acc': [0.81048095, 0.83488876, 0.8298636, 0.8743719, 0.8384781, 0.83704233, 0.8528356, 0.84350324, 0.8442211, 0.86862886], 'val_loss': [1.034171962738037, 0.893028450012207, 0.9336271643638611, 1.085134494304657, 0.9353517055511474, 1.2041330099105836, 1.1046332597732544, 0.9845291256904602, 1.0642580151557923, 1.1887439966201783], 'val_acc': [0.7077922, 0.71428573, 0.77922076, 0.72077924, 0.76623374, 0.6363636, 0.72727275, 0.8116883, 0.71428573, 0.6168831]}


In [38]:
performance = model.evaluate(x_test, y_test, batch_size=y_test.shape[0])
print("test_loss: {}, test_acc: {}".format(performance[0], performance[1]) )

312/312 [==============================] - 7s 22ms/sample - loss: 1.0957 - acc: 0.7083
test_loss: 1.095734715461731, test_acc: 0.7083333134651184


In [39]:
y_prob = model.predict(x_test) 
y_classes = y_prob.argmax(axis=-1)
y_truth = y_test.argmax(axis=-1)
breast_acc = (np.sum(y_classes[y_truth == 2] == 2)) / np.sum(y_truth == 2)
prostate_acc = (np.sum(y_classes[y_truth == 1] == 1)) / np.sum(y_truth == 1)
noncancer_acc = (np.sum(y_classes[y_truth == 0] == 0)) / np.sum(y_truth == 0)
print(" breast_acc: {} \n prostate_acc: {} \n non-cancer_acc: {}".format(breast_acc, prostate_acc, noncancer_acc))

 breast_acc: 0.8762886597938144 
 prostate_acc: 0.46153846153846156 
 non-cancer_acc: 0.7066666666666667
